In [ ]:
import pandas as pd

csv_file_path = '../DATA/RAW/dirty_cafe_sales.csv'

csv_file = pd.read_csv(csv_file_path)
nb_column = len(csv_file.columns)

dupless_csv_file = csv_file.drop_duplicates()






The number of columns is: 8
